In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import plot_model
from tensorflow.keras import metrics
import numpy as np
import time
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

#VG-Cos Method
$$C = e^{-rt}\sum_{k=0}^{N-1} \operatorname{Re} \left\{ e^{iu\mu t}\cdot \left(1- iu\theta\nu + \frac{\sigma^2\nu u^2}{2}\right)^{\frac{-t}{\nu}} \cdot e^{iux} \cdot e^{-iua} \right\} \cdot \left(  \frac{2}{b-a} K\left[\chi_{k}(0, b)-\varphi_{k}(0, b)\right] \right)$$

\begin{align}
u &= \frac{k\pi}{b-a}\\
\mu &= r-q+\frac{1}{\nu}ln\left( 1-\theta\nu-\frac{\sigma^2\nu}{2} \right)\\
x &= log\left( \frac{S_0}{K}\right)
\end{align}


\begin{align}
 \psi_{k}(c, d) =  \begin{cases}{\left[\sin \left(k \pi \frac{d-a}{b-a}\right)-\sin \left(k \pi \frac{c-a}{b-a}\right)\right] \frac{b-a}{k \pi}} & k \neq 0 \\ (d-c) & k=0 .\end{cases}
 \end{align}

\begin{align}
\chi_{k}(c, d) = \frac{1}{1+\left(\frac{k \pi}{b-a}\right)^{2}}\left[\cos \left(k \pi \frac{d-a}{b-a}\right) e^{d}-\cos \left(k \pi \frac{c-a}{b-a}\right) e^{c}\right. \left.+\frac{k \pi}{b-a} \sin \left(k \pi \frac{d-a}{b-a}\right) e^{d}-\frac{k \pi}{b-a} \sin \left(k \pi \frac{c-a}{b-a}\right) e^{c}\right]
\end{align}

In [ ]:
def CHI(a,b,c,d,k):
    A = (k*np.pi)/(b-a)
    val = (1/(1+A**2))*( np.cos(A*(d-a))*np.exp(d)- np.cos(A*(c-a))*np.exp(c) + A*np.sin(A*(d-a))*np.exp(d) - A*np.sin(A*(c-a))*np.exp(c)  )
    return val

def PSI(a,b,c,d,k):
    A = (k*np.pi)/(b-a)
    if k == 0:
        val = d-c 
    else:
        val = A**(-1)*( np.sin(A*(d-a)) - np.sin(A*(c-a)) )
    return val

def V_call(K,a,b,k): #V_k for vanilla call options
    val = (2/(b-a))*K*(CHI(a,b,0,b,k)-PSI(a,b,0,b,k))
    return val

def VG_CHARAC(S0,K,T,r,a,b,sig,theta,nu,k):
    i = complex(0,1)
    x = np.log(S0/K)
    u = (k*np.pi)/(b-a)
    mu = r + ( 1/nu*(np.log(1-theta*nu-0.5*sig**2*nu)) )
    val = np.exp(i*u*x) * np.exp(i*u*mu*T) * (1-i*u*theta*nu+0.5*sig**2*nu*u**2)**(-T/nu) #VG characteristic function
    return val

def VG_EuroCall(S0,K,T,r,sig,theta,nu,N): #pricing formula
    i = complex(0,1)
    L = 10
    c1 = (r+theta)*T
    c2 = (sig**2 + nu*theta**2)*T
    c4 = 3*(sig**4*nu + 2*theta**4*nu**3 + 4*sig**2*theta**2*nu**2)*T
    a = c1 - L*np.sqrt(c2+ np.sqrt(c4)) 
    b = c1 + L*np.sqrt(c2+ np.sqrt(c4)) 
    P = []
    for k in range(N):
        u = (k*np.pi)/(b-a)
        val = (VG_CHARAC(S0,K,T,r,a,b,sig,theta,nu,k) * np.exp(-i*a*u)).real * V_call(K,a,b,k) 
        P.append(val)
    P[0] = 0.5*P[0] #first term is weighted by half
    val = (sum(P))*np.exp(-r*T)
    return val

#Statistic Function

In [ ]:
def STAT(L):
    """
    Statistic function
    Inputs: List/array L
    Outputs: max, min, mean
    """
    print("max = "+str(np.max(L)) )
    print("min = "+str(np.min(L)) )
    print("mean = "+str(np.sum(L)/len(L)) )

#Range for strikes and maturity times
The NN is trained on $5$ stikes and $4$ maurities, thus resulting into a vector of $20$ prices

In [ ]:
S0 = 1.
K = np.array([0.8,0.9,1.0,1.1, 1.2], dtype=np.float32) #list of strikes
T = np.array([.5,.75,1.,1.25], dtype=np.float32) #list of maturities

In [ ]:
r,sig,theta,nu = 0.1,0.2,-0.14,0.2

for i in range(len(T)):
    print(VG_EuroCall(S0,K,T[i],r,sig,theta,nu,256)) #example

[0.24244133 0.15607768 0.08371659 0.03516488 0.01227804]
[0.2629346  0.18055701 0.1106998  0.05920353 0.02778986]
[0.28267395 0.2034122  0.13530204 0.0823326  0.04587126]
[0.30167663 0.22500885 0.15832527 0.10457461 0.0648732 ]


# Pricing Neural Network

In [ ]:
VG_pricer=Sequential([
    Dense(16, activation ="relu", input_shape=(4,)),
    Dense(64, activation ="relu"),
    Dense(64, activation ="relu"),
    Dense(32, activation ="relu"),
    Dense(len(K)*len(T)) ])

lr = 0.02
loss = "mse"

#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
optimizer = tf.keras.optimizers.SGD( learning_rate = lr, momentum= 0.8, nesterov=False)

VG_pricer.compile(loss = loss, optimizer = optimizer,metrics = ['accuracy'])
VG_pricer.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 64)                1088      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 20)                660       
                                                                 
Total params: 8,068
Trainable params: 8,068
Non-trainable params: 0
_________________________________________________________________


#Training
* For each training epoch a new data set is created. This is time cosuming, but it avoids over fitting.
* use random values of $\{r,\sigma,\theta,\nu\}$ for each set, such that;
\begin{align}
r &= [0,0.2]\\
\sigma &= [0.0,0.2]\\
\theta &= [-0.3,0.05]\\
\nu &= [0.0,0.7]\\
\end{align}

#Since we alrady have a pre-trained model we skip the training process

In [ ]:
Training_epochs = 100 #Training epochs
Data_size = 1024 #Training Data Size per epoch

Y_mat = np.zeros((Data_size, len(T),len(K))) 

for n in range(Training_epochs):
    r = np.random.uniform(low = 0.0, high = 0.2, size = (Data_size,1) ) 
    sig = np.random.uniform(low = 0.0, high = 0.2,size = (Data_size,1) )
    theta = np.random.uniform(low = -0.3, high = 0.05,size = (Data_size,1) )
    nu = np.random.uniform(low = 0.0, high = 0.7,size = (Data_size,1) )
    X=np.concatenate([r,sig,theta,nu],axis=1) #Inputs

    for b in range(Data_size):
        for i in range(len(T)):
            Y_mat[b,i,:]= VG_EuroCall(S0,K,T[i],r[b],sig[b],theta[b],nu[b],256)
    Y = Y_mat.reshape((-1,len(K)*len(T))) #Targets. 20 option prices for each vector [r,sig,theta,nu]
    
    if n%5==0:
        lr = lr*0.95  #Decreases the learnig rate after every 5 epochs
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
        #optimizer = tf.keras.optimizers.SGD( learning_rate = lr, momentum= 0.8, nesterov=False)
        VG_pricer.compile(loss = loss, optimizer = optimizer,metrics=['accuracy'])

    if n == int(Training_epochs/2): #Training marker for half the epochs
         print("half way point")

    history = VG_pricer.fit(X,Y, epochs = 1, batch_size = 64) #NN Training

#Evaluation on unseen data

In [ ]:
val_data_size = 2048

r_val = np.random.uniform(low = 0, high = 0.2, size = (val_data_size,1) )
sig_val = np.random.uniform(low = 0.0, high = 0.25,size = (val_data_size,1) )
theta_val = np.random.uniform(low =-0.3, high = 0.06,size = (val_data_size,1) )
nu_val = np.random.uniform(low = 0.0, high = 0.8,size = (val_data_size,1) )

X_val = np.concatenate([r_val,sig_val,theta_val,nu_val],axis=1) #inputs
Y_mat_val = np.zeros((val_data_size, len(T), len(K)))

for b in range(val_data_size):
    for i in range(len(T)):
        Y_mat_val[b,i,:] = VG_EuroCall(S0,K,T[i],r_val[b],sig_val[b],theta_val[b],nu_val[b],256)
        
Y_val = Y_mat_val.reshape((-1,len(K)*len(T))) #Target. 20 0ption prices

VG_pricer.evaluate(X_val, Y_val)

64/64 [==============================] - 0s 2ms/step - loss: 5.8166e-05 - accuracy: 0.9888


[5.816601333208382e-05, 0.98876953125]

#Comparing predictions against actual prices

In [ ]:
r = np.random.uniform(low = 0, high = 0.2)
sig = np.random.uniform(low = 0.0, high = 0.2)
theta = np.random.uniform(low =-0.3, high = 0.05 )
nu = np.random.uniform(low = 0.0, high = 0.7 )

parameters = [r,sig,theta,nu]

y_pred = VG_pricer.predict([parameters]) #Using the pricing NN
y_act = np.zeros((len(T),len(K)))

for i in range(len(T)):
        y_act[i,:] = VG_EuroCall(S0,K,T[i],r,sig,theta,nu,256)
y_pred = np.array(tf.reshape(y_pred, (4,5)))

print(parameters,"parameters","\n"*2)
print(y_pred,"Prediction",2*"\n",y_act,"Actual")

[0.0976376409766925, 0.09549363196110922, -0.26898386008162517, 0.4061044729415425] parameters 


[[0.24445173 0.15742652 0.08245751 0.02426253 0.00210783]
 [0.26498717 0.18172842 0.10668307 0.05060683 0.0168573 ]
 [0.28757295 0.20596087 0.13476166 0.07303146 0.03555968]
 [0.30221215 0.22811566 0.15662052 0.09683903 0.05557131]] Prediction 

 [[0.24288301 0.15606685 0.0800299  0.0241871  0.00094391]
 [0.2629399  0.17972505 0.10647202 0.04908726 0.01341894]
 [0.28215906 0.20182846 0.13057445 0.07257487 0.0314981 ]
 [0.30063054 0.22275127 0.15313812 0.09494386 0.05077351]] Actual


#Relative pricing error for 20 options as a percentage.

In [ ]:
L = (np.abs(y_pred-y_act)/y_act)*100
print(L,"\n")
L = np.array( tf.reshape(L,(1,20)) )[0]
STAT(L)

[[  0.64587463   0.87121087   3.03337581   0.31184373 123.30917208]
 [  0.77860779   1.11468697   0.19822354   3.09565477  25.62311561]
 [  1.91873686   2.0474837    3.20676166   0.62912478  12.89468135]
 [  0.52609733   2.40824132   2.27402652   1.99610002   9.44941788]] 

max = 123.30917207828746
min = 0.1982235386585934
mean = 9.816621859727979


# Saving the Pricng NN

In [ ]:
#VG_pricer.save(Variance_Gamma_Pricing_NN_Weights.h5')

#For calibration, we load a pre-trained model

In [ ]:
VG_pricer = load_model('Variance_Gamma_Pricing_NN_Weights.h5') 

In [ ]:
S0 = 1.
K = np.array([0.8,0.9,1.0,1.1, 1.2], dtype=np.float32)
T = np.array([.5,.75,1.,1.25], dtype=np.float32)

# Constructing the calibrator
* To make the calibrator more stable the actiation function used on the first hidden layer is custom made to be within the range of each parameter of interest, and this is achieved through modifying the sigmoid function to produce outputs in the range $[a,b]$ rather than its original range of $[0,1]$. In the modified sigmoid, $c$ marks where the function is centered and $d$ controls the slope of the graph.

In [ ]:
def modified_sigmoid(x,a = 0.0,b = 1.0,c = 0.0,d = 1.0): #Modified sigmoid
    return (b-a)*tf.math.sigmoid( (x-c)/d ) + a

@tf.function 
def Custom_Activation(x): #custom activation function
    return tf.stack([modified_sigmoid(x[...,0],0.0,0.2,0.1,0.05), modified_sigmoid(x[...,1],0.0,0.2,0.1,0.05),
                     modified_sigmoid(x[...,2],-0.3,0.05,-.1525,0.05), modified_sigmoid(x[...,3],0.0,0.7,.35,0.05) ], axis = 1)

* The calibrator NN is set up in such a way that the parameters being trained are the weights from the first hidden layer, thus the bias from this layer is deactivated. Hidden layer after the first hidden layer are the same as in the pricing NN as they use weights obtained from it.

In [ ]:
VG_calibrator = Sequential([
    Dense(4, input_dim= 1, use_bias=False , kernel_initializer = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),
          activation = Custom_Activation),
    Dense(16, activation ="relu"),
    Dense(64, activation ="relu"),
    Dense(64, activation ="relu"),
    Dense(32, activation ="relu"),
    Dense(len(K)*len(T))    ]) 

#setting and fixing the weights obtained from the pricing NN
for i in range(1, len(VG_calibrator.layers)):
    VG_calibrator.layers[i].set_weights(VG_pricer.layers[i-1].weights)
    VG_calibrator.layers[i].trainable=False 
VG_calibrator.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 4)                 4         
                                                                 
 dense_6 (Dense)             (None, 16)                80        
                                                                 
 dense_7 (Dense)             (None, 64)                1088      
                                                                 
 dense_8 (Dense)             (None, 64)                4160      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 20)                660       
                                                                 
Total params: 8,072
Trainable params: 4
Non-trainable 

* The calibrator NN takes the vector of $[1]$ as an input and learns the weights of the fist hidden layer by matching them to actual prices on the market, thus we create a set of 20 prices to represent the market observed prices

In [ ]:
r = np.random.uniform(low = 0, high = 0.2 )
sig = np.random.uniform(low = 0.0, high = 0.2 )
theta = np.random.uniform(low =-0.3, high = 0.05)
nu = np.random.uniform(low = 0.0, high = 0.7)

actual_parameters = np.array([[r,sig,theta,nu]])  #Paramerts to calibrate
y_mrkt = VG_pricer(actual_parameters).numpy() #Represents market observed prices

* The calibration may need to be run 2 or 3 times in order to improve the accuarcy of the model

In [ ]:
def scheduler(epoch, lr): #learning rate schedular
      if epoch % 10==0:
        return lr*0.95
      else:
        return lr 
    
for _ in range(8): #training the calibrator
    optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    VG_calibrator.compile(loss = "mse" , optimizer = optimizer,metrics = ["accuracy"])
    callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
    history = VG_calibrator.fit(np.array([[1.]]), y_mrkt, epochs = 100, verbose = 0, callbacks = [callback])

w = VG_calibrator.layers[0].weights[0].numpy() #exatracting the first layer weights
pars_pred = Custom_Activation(w).numpy()  #passing the weight throughh the custom activation

print(2*"\n",pars_pred,"Prediction""\n",actual_parameters,"Actual") #Predicted vs actual parameters



 [[ 0.09120898  0.17064026 -0.0181556   0.21399993]] Prediction
 [[ 0.09126296  0.17117333 -0.01864222  0.22158758]] Actual
